In [ ]:
# !pip uninstall llvmlite -y

In [ ]:
!pip install --ignore-installed -U mljar-supervised

In [ ]:
import os
import gc
import time
import math
import random
import numpy as np
import pandas as pd

from supervised.automl import AutoML

import warnings
warnings.simplefilter('ignore')

def reduce_memory_usage(df, verbose=True):
    numerics = ["int8", "int16", "int32", "int64", "float16", "float32", "float64"]
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if (
                    c_min > np.finfo(np.float16).min
                    and c_max < np.finfo(np.float16).max
                ):
                    df[col] = df[col].astype(np.float16)
                elif (
                    c_min > np.finfo(np.float32).min
                    and c_max < np.finfo(np.float32).max
                ):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print(
            "Mem. usage decreased to {:.2f} Mb ({:.1f}% reduction)".format(
                end_mem, 100 * (start_mem - end_mem) / start_mem
            )
        )
    return df

In [ ]:
train_data = pd.read_csv('../input/tabular-playground-series-nov-2021/train.csv')
test_data = pd.read_csv('../input/tabular-playground-series-nov-2021/test.csv')


idx = test_data.index

train_data = reduce_memory_usage(train_data)
test_data = reduce_memory_usage(test_data)

# prepare dataframe for modeling
X = train_data.drop(columns=['target'])
y = train_data['target']

In [ ]:
train=train_data.copy()
test=test_data.copy()
unique_values = train.iloc[:1000].nunique() < 0
categoricals = [col for col in  unique_values.index[unique_values < 10] if col!='target']
numeric = [col for col in test.columns if col not in categoricals]

train['mean_numeric'] = train[numeric].mean(axis=1)
train['std_numeric'] = train[numeric].std(axis=1)
train['min_numeric'] = train[numeric].min(axis=1)
train['max_numeric'] = train[numeric].max(axis=1)
train['sum_categoricals'] = train[categoricals].sum(axis=1)

test['mean_numeric'] = test[numeric].mean(axis=1)
test['std_numeric'] = test[numeric].std(axis=1)
test['min_numeric'] = test[numeric].min(axis=1)
test['max_numeric'] = test[numeric].max(axis=1)
test['sum_categoricals'] = test[categoricals].sum(axis=1)

In [ ]:
del train_data

In [ ]:
gc.collect()

In [ ]:
automl = AutoML(mode="Compete",
                results_path="./AutoML_classifier_5_all",
                eval_metric='auc',
                algorithms=["CatBoost", "Xgboost", "LightGBM", "Random Forest", "Extra Trees", "Neural Network"],
                total_time_limit=3*3600,
                validation_strategy={
                    "validation_type": "kfold",
                    "k_folds": 5,
                    "shuffle": True,
                    "stratify": True,
                    "random_seed": 42
                })

# train models with AutoML
automl.fit(X, y)
automl.report()

In [ ]:
# Use the model to generate predictions
predictions = automl.predict_proba(test_data)

submission = pd.read_csv('../input/tabular-playground-series-nov-2021/sample_submission.csv')

submission['target'] = predictions[:, -1]
submission.to_csv("submission.csv", index=False)
submission.head()